In [ ]:
gold_path = '<>'

#### Approach #1 - sale_by_date_city

In [ ]:
df_fact_sale = spark.read.table("default.silver_fact_sale") 
df_dimension_date = spark.read.table("default.silver_dimension_date")
df_dimension_city = spark.read.table("default.silver_dimension_city")

In [ ]:
sale_by_date_city = df_fact_sale.alias("sale") \
.join(df_dimension_date.alias("date"), df_fact_sale.InvoiceDateKey == df_dimension_date.Date, "inner") \
.join(df_dimension_city.alias("city"), df_fact_sale.CityKey == df_dimension_city.CityKey, "inner") \
.select("date.Date", "date.CalendarMonthLabel", "date.Day", "date.ShortMonth", "date.CalendarYear", "city.City", "city.StateProvince", "city.SalesTerritory", "sale.TotalExcludingTax", "sale.TaxAmount", "sale.TotalIncludingTax", "sale.Profit")\
.groupBy("date.Date", "date.CalendarMonthLabel", "date.Day", "date.ShortMonth", "date.CalendarYear", "city.City", "city.StateProvince", "city.SalesTerritory")\
.sum("sale.TotalExcludingTax", "sale.TaxAmount", "sale.TotalIncludingTax", "sale.Profit")\
.withColumnRenamed("sum(TotalExcludingTax)", "SumOfTotalExcludingTax")\
.withColumnRenamed("sum(TaxAmount)", "SumOfTaxAmount")\
.withColumnRenamed("sum(TotalIncludingTax)", "SumOfTotalIncludingTax")\
.withColumnRenamed("sum(Profit)", "SumOfProfit")\
.orderBy("date.Date", "city.StateProvince", "city.City")

table_name = 'gold_aggregate_sale_by_date_city'
sale_by_date_city.write.mode("overwrite").format("delta").option("path", gold_path + table_name).saveAsTable("gold_" + table_name)

#### Approach #2 - sale_by_date_employee

In [ ]:
%sql
CREATE OR REPLACE TEMPORARY VIEW sale_by_date_employee
AS
SELECT
	DD.Date, DD.CalendarMonthLabel
    , DD.Day, DD.ShortMonth Month, CalendarYear Year
	,DE.PreferredName, DE.Employee
	,SUM(FS.TotalExcludingTax) SumOfTotalExcludingTax
	,SUM(FS.TaxAmount) SumOfTaxAmount
	,SUM(FS.TotalIncludingTax) SumOfTotalIncludingTax
	,SUM(Profit) SumOfProfit 
FROM default.silver_fact_sale FS
INNER JOIN default.silver_dimension_date DD ON FS.InvoiceDateKey = DD.Date
INNER JOIN default.silver_dimension_Employee DE ON FS.SalespersonKey = DE.EmployeeKey
GROUP BY DD.Date, DD.CalendarMonthLabel, DD.Day, DD.ShortMonth, DD.CalendarYear, DE.PreferredName, DE.Employee
ORDER BY DD.Date ASC, DE.PreferredName ASC, DE.Employee ASC

In [ ]:
sale_by_date_employee = spark.sql("SELECT * FROM sale_by_date_employee")

table_name = 'gold_aggregate_sale_by_date_employee'
sale_by_date_employee.write.mode("overwrite").format("delta").option("path", gold_path + table_name).saveAsTable("gold_" + table_name)

In [ ]:
spark.catalog.listTables()